In [ ]:
import subprocess, os, sys, time, importlib

os.chdir('/workspaces/scrapermeleegg')
sys.path.insert(0, '/workspaces/scrapermeleegg')

# Kill any existing process on port 8080
subprocess.run('kill $(lsof -t -i:8080) 2>/dev/null', shell=True)
time.sleep(1)

# Regenerate the site fresh (reload modules to pick up all changes)
import generate_site
importlib.reload(generate_site)
import manage_tournaments as mt
importlib.reload(mt)

db = mt.TournamentDB()
dist_path = '/workspaces/scrapermeleegg/dist'
if db.tournaments:
    generate_site.generate_multi_tournament_site(db.data['tournaments'], dist_path)
    size = os.path.getsize(os.path.join(dist_path, 'index.html'))
    print(f'Site generated: {size} bytes')

# Check HTML contents
with open(os.path.join(dist_path, 'index.html'), 'r') as f:
    html = f.read()
for term in ['home-panel', 'add-box', 'Scrapear', 'btn-do-add', 'nav-home', 'Torneos']:
    print(f'  {term}: {html.count(term)}')

# Launch server with updated no-cache headers
server = subprocess.Popen(
    [sys.executable, 'server.py', '--port', '8080'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
    cwd='/workspaces/scrapermeleegg'
)
time.sleep(2)
print(f'Server PID: {server.pid}')

# Verify server is responding  
r = subprocess.run('curl -sI http://localhost:8080/ | head -15', shell=True, capture_output=True, text=True)
print(f'Response headers:\n{r.stdout}')
print('Server running at http://localhost:8080')

[*] Procesando: Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am
    Arquetipos con partidas: 104

[✓] Sitio generado en: /workspaces/scrapermeleegg/dist/
    index.html  (737,861 bytes)
    Torneos embebidos: 1
      • Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am (5046 matches)
Site generated: 740368 bytes
  home-panel: 6
  add-box: 6
  Scrapear: 2
  btn-do-add: 4
Server PID: 41122
HTTP response: 200
Server running at http://localhost:8080
